In [ ]:
import sys
import subprocess

#import pkg_resources
#required = {'harmonypy','sklearn','scanpy','pandas', 'numpy', 'bbknn', 'scipy', 'matplotlib', 'seaborn' ,'scipy'}
#installed = {pkg.key for pkg in pkg_resources.working_set}
#missing = required - installed
#if missing:
#    print("Installing missing packages:" )
#    print(missing)
#    python = sys.executable
#    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

%matplotlib inline
from collections import Counter
from collections import defaultdict
import scanpy as sc
import pandas as pd
import pickle as pkl
import numpy as np
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys
from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns; sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
import harmonypy as hm
from pathlib import Path

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')

In [2]:
# Introduce variables
# Note that this script expects raw data to be in "non-batch-corrected" adata.raw.X. 

save_path = '/Users/emilyjones/Desktop/HCA/project/script templates/mouse/mouse_exp1_exp2_concat.h5ad'

# Required: Name of first object
data1 = "/Users/emilyjones/Desktop/HCA/project/ex_mat/exp_1_ortholog_deleted_rows.csv"
# Provide path to obj1 // landscape/training data
Object1 = "/Users/emilyjones/Desktop/HCA/project/script templates/experiment_1.h5ad"

# Required: Name of second object
data2 = "/Users/emilyjones/Desktop/HCA/project/ex_mat/exp_2_ortholog_deleted_rows.csv"
# Provide path to obj2 // prediction/projection data
Object2 = "/Users/emilyjones/Desktop/HCA/project/script templates/experiment_2.h5ad"

# Optional: Batch correction options (this is for correction of eventual combined dataset for data1 and data2
batch_correction = True
batch = ["batch_id","batch_id",] # Will accept any batch categorical. Comma space a batch categorical for each dataset. Position 1 is for data1, position 2 is for data2

In [3]:
%%time
# Check if filepaths are good
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
if (Path(Object1).is_file() & Path(Object2).is_file()):
    print("adata file paths detetcted, proceeding to load")
    adata = sc.read(Object1)
    adata2 =  sc.read(Object2)
    del adata.uns
    del adata2.uns
else: 
    raise TypeError("one or more .h5ad paths cannot be accessed")

# altering scanpy setting so that we can save it to our defined directory
sc._settings.ScanpyConfig(figdir=save_path)

# Combine and pre-process data to match correlations across PCA

adata file paths detetcted, proceeding to load
CPU times: user 124 ms, sys: 29.9 ms, total: 154 ms
Wall time: 205 ms


In [21]:
adata.shape

(20, 10957)

In [6]:
adata2.shape

(8, 10957)

In [7]:
adata.obs.index + '_'

Index(['CCR1_R1_', 'CCR1_R2_', 'CCR1_R3_', 'CCR2_R1_', 'CCR2_R2_', 'CCR2_R3_',
       'CCR5_R1_', 'CCR5_R2_', 'CCR5_R3_', 'NO_CCR_R1_', 'NO_CCR_R2_',
       'NO_CCR_R3_'],
      dtype='object', name='index')

In [8]:
adata2.obs.index + '_'

Index(['CCR1_R1_', 'CCR1_R2_', 'CCR1_R3_', 'CCR1_R4_', 'CCR2_R1_', 'CCR2_R2_',
       'CCR2_R3_', 'CCR2_R4_'],
      dtype='object', name='index')

In [9]:
adata.obs['celltype'] = 'macropahge'
adata.obs['id'] =  adata.obs.index + '_' + adata.obs['celltype']
adata.obs.index = adata.obs.index + '_' + adata.obs['experiment_id'].astype(str)
adata.obs['batch_id'] = adata.obs.index.str.replace('_R.*_exp_1', '')
adata.obs['batch_id'] = adata.obs['batch_id'].astype(str) + '_' + adata.obs['experiment_id'].astype(str)

<ipython-input-9-6a93cb8b77ef>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  adata.obs['batch_id'] = adata.obs.index.str.replace('_R.*_exp_1', '')


In [10]:
adata.obs

,experiment_id,celltype,id,batch_id
CCR1_R1_exp_1,exp_1,macropahge,CCR1_R1_macropahge,CCR1_exp_1
CCR1_R2_exp_1,exp_1,macropahge,CCR1_R2_macropahge,CCR1_exp_1
CCR1_R3_exp_1,exp_1,macropahge,CCR1_R3_macropahge,CCR1_exp_1
CCR2_R1_exp_1,exp_1,macropahge,CCR2_R1_macropahge,CCR2_exp_1
CCR2_R2_exp_1,exp_1,macropahge,CCR2_R2_macropahge,CCR2_exp_1
CCR2_R3_exp_1,exp_1,macropahge,CCR2_R3_macropahge,CCR2_exp_1
CCR5_R1_exp_1,exp_1,macropahge,CCR5_R1_macropahge,CCR5_exp_1
CCR5_R2_exp_1,exp_1,macropahge,CCR5_R2_macropahge,CCR5_exp_1
CCR5_R3_exp_1,exp_1,macropahge,CCR5_R3_macropahge,CCR5_exp_1
NO_CCR_R1_exp_1,exp_1,macropahge,NO_CCR_R1_macropahge,NO_CCR_exp_1


In [11]:
adata2.obs['celltype'] = 'mono'
adata2.obs['id'] =  adata2.obs.index + '_' + adata2.obs['celltype']
adata2.obs.index = adata2.obs.index + '_' + adata2.obs['experiment_id'].astype(str)
adata2.obs['batch_id'] = adata2.obs.index.str.replace('_R.*_exp_2', '')
adata2.obs['batch_id'] = adata2.obs['batch_id'].astype(str) + '_' + adata2.obs['experiment_id'].astype(str)

<ipython-input-11-8cdfc5ff7a7f>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  adata2.obs['batch_id'] = adata2.obs.index.str.replace('_R.*_exp_2', '')


In [12]:
adata2.obs

,experiment_id,celltype,id,batch_id
CCR1_R1_exp_2,exp_2,mono,CCR1_R1_mono,CCR1_exp_2
CCR1_R2_exp_2,exp_2,mono,CCR1_R2_mono,CCR1_exp_2
CCR1_R3_exp_2,exp_2,mono,CCR1_R3_mono,CCR1_exp_2
CCR1_R4_exp_2,exp_2,mono,CCR1_R4_mono,CCR1_exp_2
CCR2_R1_exp_2,exp_2,mono,CCR2_R1_mono,CCR2_exp_2
CCR2_R2_exp_2,exp_2,mono,CCR2_R2_mono,CCR2_exp_2
CCR2_R3_exp_2,exp_2,mono,CCR2_R3_mono,CCR2_exp_2
CCR2_R4_exp_2,exp_2,mono,CCR2_R4_mono,CCR2_exp_2


In [13]:
# Define intersecting genes between datasets
adata_genes = list(adata.var.index)
adata2_genes = list(adata2.var.index)
keep_SC_genes = list(set(adata_genes) & set(adata2_genes))
print("keep gene list = " , len(keep_SC_genes), "adata1 gene length = ", len(adata_genes) , "adata2 gene length = ", len(adata2_genes) )

# Remove non-intersecting genes (this step will remove cite-seq data if training data is pure RNA seq)
adata_intersect1 = adata[:, keep_SC_genes]
adata = adata_intersect1
adata_intersect2 = adata2[:, keep_SC_genes]
adata2 = adata_intersect2

keep gene list =  10957 adata1 gene length =  13665 adata2 gene length =  10957


In [14]:
# Create a common batch column and do simple sanity check for batch variables
if not((batch_correction == "False") and (len(batch)>1)):
    print("Batch correction option detected, proceeding to format batch variables")
    batch_var = "lr_batch"
    adata.obs["lr_batch"] = adata.obs[batch[0]]
    adata2.obs["lr_batch"] = adata2.obs[batch[1]]
else: raise TypeError("Batch correction option detected but requires at least one categorical for each dataset!")

# Create a common obs column in both datasets containing the data origin tag
common_cat = "corr_concat" 
adata.obs = adata.obs.astype('category')
adata2.obs = adata2.obs.astype('category')
concat = adata2.concatenate(adata, join='inner',index_unique=None, batch_categories=None)
adata = concat[:]

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


Batch correction option detected, proceeding to format batch variables


In [15]:
adata.write(save_path)

/Applications/anaconda3/envs/python_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/Applications/anaconda3/envs/python_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'experiment_id' as categorical
/Applications/anaconda3/envs/python_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True

In [16]:
adata.obs

,experiment_id,celltype,id,batch_id,lr_batch,batch
CCR1_R1_exp_2,exp_2,mono,CCR1_R1_mono,CCR1_exp_2,CCR1_exp_2,0
CCR1_R2_exp_2,exp_2,mono,CCR1_R2_mono,CCR1_exp_2,CCR1_exp_2,0
CCR1_R3_exp_2,exp_2,mono,CCR1_R3_mono,CCR1_exp_2,CCR1_exp_2,0
CCR1_R4_exp_2,exp_2,mono,CCR1_R4_mono,CCR1_exp_2,CCR1_exp_2,0
CCR2_R1_exp_2,exp_2,mono,CCR2_R1_mono,CCR2_exp_2,CCR2_exp_2,0
CCR2_R2_exp_2,exp_2,mono,CCR2_R2_mono,CCR2_exp_2,CCR2_exp_2,0
CCR2_R3_exp_2,exp_2,mono,CCR2_R3_mono,CCR2_exp_2,CCR2_exp_2,0
CCR2_R4_exp_2,exp_2,mono,CCR2_R4_mono,CCR2_exp_2,CCR2_exp_2,0
CCR1_R1_exp_1,exp_1,macropahge,CCR1_R1_macropahge,CCR1_exp_1,CCR1_exp_1,1
CCR1_R2_exp_1,exp_1,macropahge,CCR1_R2_macropahge,CCR1_exp_1,CCR1_exp_1,1


In [18]:
adata.shape

(20, 10957)